<a href="https://colab.research.google.com/github/ucguate/climahub-etl/blob/ElSalvador/ETL_climahub_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Psycopg2 module installation
!pip install psycopg2
!pip install requests

In [7]:
# importing psycopg2 
import psycopg2 
from psycopg2 import Error
from pprint import pprint
# import requests for rest requests 
import requests
# importing json module
import json
# importing pandas to manage Dictionaries as pandas DataFrames
import pandas as pd
import re 

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [59]:
# manipulate data functions
def get_estaciones(option = "JSON"):
  queryParams = {
      'option': option,
      'key': "098f6bcd4621d373cade4e832627b4f6",
  }
  response = requests.get("https://www.snet.gob.sv/dataservices/getEstaciones.php", params=queryParams)
  #print(response.json)
  data = json.loads(response.text)
  #data = data["sensores"]
  #pprint(data)

  #getting column names
  #variables = data["features"][1].keys()
  #print(variables)

  df = pd.DataFrame(data)
  df.rename(columns = {0:'id',1:'name',2:'lat',3:'lon',}, inplace = True)
  #display(df)
  return df;

# manipulate data functions
def get_data(option = "JSON"):
  queryParams = {
      'option': option,
      'key': "098f6bcd4621d373cade4e832627b4f6",
  }
  response = requests.get("https://www.snet.gob.sv/dataservices/getData.php", params=queryParams)
  #print(response.json)
  data = json.loads(response.text)
  #data = data["sensores"]
  #pprint(data)

  #getting column names
  #variables = data["features"][1].keys()
  #print(variables)

  df = pd.DataFrame(data)
  df.rename(columns = {0:'id_estacion',1:'param',2:'datetime',3:'valor',}, inplace = True)
  #display(df)
  return df;

# manipulate data functions
def get_parametros(option = "JSON"):
  queryParams = {
      'option': option,
      'key': "098f6bcd4621d373cade4e832627b4f6",
  }
  response = requests.get("https://www.snet.gob.sv/dataservices/getParametros.php", params=queryParams)
  #print(response.json)
  data = json.loads(response.text)
  #data = data["sensores"]
  #pprint(data)

  #getting column names
  #variables = data["features"][1].keys()
  #print(variables)

  df = pd.DataFrame(data)
  #df.rename(columns = {0:'id',1:'name',2:'lat',3:'lon',}, inplace = True)
  #display(df)
  return df;

In [35]:
dfp = get_parametros()
dfp

,0,1,2
0,PC,Precipitacion Acumulada,mm
1,PP,Precipitacion Instantanea,mm
2,HG,Nivel del Rio,m
3,AT,Temperatura del aire,°C
4,BP,Presion Barometrica,milibares
5,DA,Dir. del Viento (Prom),Grados
6,DI,Dir. del Viento (Inst),Grados
7,DR,Dir. de Rafaga Diaria,Grados
8,HR,Luz Solar,Hrs
9,LH,Humedad Relativa (Min),%


# DATABASE CONNECTION and Insert Queries


In [109]:
# creating querying functions
def show_tables(cursor):
    query = """
      SELECT *
      FROM pg_catalog.pg_tables
      WHERE schemaname != 'pg_catalog' AND 
          schemaname != 'information_schema';
    """
    cursor.execute(query)
    pprint(cursor.fetchall())

def get_table_cols(table_name):
    query = """
    SELECT column_name
      FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name   = '{}'
     ;
    """.format(table_name)

    cursor.execute(query)
    data = cursor.fetchall()
    pprint(data)
  
def station_columns(): 
  query2 = """
    SELECT column_name
      FROM information_schema.columns
        WHERE table_schema = 'public'
          AND table_name   = '{}'
     ;
    """.format('station')
  cursor.execute(query2)
  pprint(cursor.fetchall())

def insert_station(station_name, latitude, longitude, source_station_id, original_id):
  source_id = 4 # EL Salvador
  query = """
    insert into station (source_id,station_name,latitude,longitude, source_station_id, original_id) 
      values ( '{}', '{}', '{}', '{}', '{}', '{}') on conflict do nothing;
    """.format(source_id, station_name, latitude, longitude, source_station_id, original_id)
  return query
  #cursor.execute(query)
  #pprint(cursor.fetchall())

def insert_data_all(variable_id, data_date, data_value, source_station_id):
  query = """
    INSERT INTO data_all(variable_id,data_date,data_value, station_id)
      VALUES({},'{}','{}',{}) on conflict do nothing;
    """.format(
        variable_id, data_date, data_value,
        "(SELECT id FROM station WHERE source_station_id = '{}')".format(source_station_id)
        )
  return query


In [110]:
connection = None
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="climahub",
                                  password="climahub$2021",
                                  host="138.197.98.178",
                                  port="5432",
                                  database="climahub")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("🟢 CONNECTION SUCCESFUL: PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

    show_tables(cursor)

except (Exception, Error) as error:
    print("❌ Error while connecting to PostgreSQL", error)

"""
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")"""

🟢 CONNECTION SUCCESFUL: PostgreSQL server information
{'user': 'climahub', 'dbname': 'climahub', 'host': '138.197.98.178', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.2 (Debian 13.2-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 

[('public', 'alert_types', 'climahub', None, False, False, False, False),
 ('public', 'user_groups', 'climahub', None, False, False, False, False),
 ('public', 'users', 'climahub', None, True, False, False, False),
 ('public', 'source', 'climahub', None, True, False, True, False),
 ('public', 'station', 'climahub', None, True, False, True, False),
 ('public', 'freq', 'climahub', None, True, False, True, False),
 ('public', 'variable', 'climahub', None, True, False, True, False),
 ('public', 'data_all', 'climahub', None, True, False, True, False),
 ('public', 'data_last', 'climahub'

'\nfinally:\n    if (connection):\n        cursor.close()\n        connection.close()\n        print("PostgreSQL connection is closed")'

# Data manipulation and conversion

In [45]:
dfES = get_estaciones();
dfES.set_index("id", inplace=True)
dfES

,name,lat,lon
id,,,
246,Acajutla Marea,13.57384,-89.837845
248,Piedra Cargada,14.184292,-89.516655
144,LALIBERTADMAREA,14.09999,-89.417
249,Ciudad Arce,13.829981,-89.437325
252,Nuevo Cuscatlan,13.621025,-89.263985
...,...,...,...
112,Nueva Esparta,13.787,-87.8354
264,Zoologico,13.6836,-89.1948
255,El Naranjo,13.697,-89.938


In [103]:
dfDA = get_data();
dfDA

,id_estacion,param,datetime,valor
0,246,,2021-06-30 18:37:00,12.3
1,246,MA,2021-06-30 18:37:00,7244
2,144,,2021-06-30 18:37:00,29.2
3,144,MA,2021-06-30 18:37:00,7280
4,247,,2021-06-30 18:37:00,3284
...,...,...,...,...
14806,144,,2021-06-30 22:31:00,26.8
14807,144,MA,2021-06-30 22:31:00,6343
14808,144,,2021-06-30 22:32:00,26.9
14809,144,MA,2021-06-30 22:32:00,6413


In [104]:
variables = {
  'PC': ["PCP", 5],
  'PP': ["PP", 17],
  'HG': ["LEV", 13],
  'AT': ["TMP", 3],
  'BP': ["PRS", 10], 
  'DA': ["WND", 7],
  #'DI': ["PCP", 5],
  #'DR': ["PCP", 5],
  'HR': ["HSOL", 14],
  #'LH': ["HRP", 15],
  #'LT': ["PCP", 5],
  'RA': ["WNG", 9],
  #'RD': ["PCP", 5],
  'RH': ["HRP", 15],
  'SA': ["WNS", 8],
  #'SI': ["PCP", 5],
  'SR': ["RSOL", 16],
  #'UH': ["PCP", 5],
  #'UT': ["PCP", 5],
  #'BA': ["PCP", 5],
  'DP': ["DWP", 4],
  #'QI': ["PCP", 5],
  #'PD': ["PCP", 5],
  #'RI': ["PCP", 5],
  #'SO': ["PCP", 5],
  #'MA': ["PCP", 5],
  #'SM': ["PCP", 5],
  #'LW': ["PCP", 5],
  # 'ST': ["PCP", 5],
}
variables["DP"][1]

4

In [111]:
GRAND_QUERY = ''
for index, row in dfES.iterrows():
    #print(index,' - ',row['name'], '(', row['lat'],', ',row['lon'], ')' )
    GRAND_QUERY+=insert_station(row['name'], row['lat'], row['lon'], index, index)

#print(GRAND_QUERY)
cursor.execute(GRAND_QUERY)
connection.commit()

GRAND_QUERY = ''
for index, row in dfDA.iterrows():
  if(row['param'] != "  " and row['param'] in variables):
    #print(index,' - ',row['id_estacion'],' - ',row['param'],' - ',row['datetime'],' - ',row['valor'])
    GRAND_QUERY+=insert_data_all(variables[row['param']][1],row['datetime'], row['valor'],row['id_estacion'] )
    
#print(GRAND_QUERY)
cursor.execute(GRAND_QUERY)
connection.commit()

cursor.close()
connection.close()
print("PostgreSQL connection is closed")

PostgreSQL connection is closed


Functions

In [43]:
def convert_date(date):
  #print(date)
  if(date[1:2] == '/'):
    date='0'+date
  if(date[17:19] == "AM"):
    #print("AM")
    time = date[11:16]+":00"
  else:
    #print("PM")
    hour = int(date[11:13])
    if(hour == 12):
      time = str(hour)+date[13:16]+":00"
    else:
      time = str(hour+12)+date[13:16]+":00"

  return date[6:10]+'-'+date[3:5]+'-'+date[0:2]+' '+time

def clean_valor(valor):
  valor = re.sub(r"watt/m2| |/|m/s|ºC|mbar|msnm|m|%|°", '', valor)
  return valor

# INSERT

In [46]:
# iterating recognized variables to look for into df
GRAND_QUERY = ''
for key, value in dfES.items():
  print(key,value)
  #GRAND_QUERY+=insert_station(v["nombre"], v["latitud"], v["longitud"], v["numero_estacion"], k)
  
  #iterating across "estaciones"
  #for k, v, in cur_row["estaciones"].items():
  #  print(k,v)
    
  #  GRAND_QUERY+=insert_data_all(value[1], convert_date(v["sensor_fecha"]), clean_valor(v["sensor_valor"]), v["numero_estacion"])
    
  #cursor.execute(GRAND_QUERY)
  #connection.commit()
print(GRAND_QUERY);

cursor.close()
connection.close()
print("PostgreSQL connection is closed")

name id
246     Acajutla Marea
248     Piedra Cargada
144    LALIBERTADMAREA
249        Ciudad Arce
252    Nuevo Cuscatlan
            ...       
112      Nueva Esparta
264          Zoologico
255         El Naranjo
257           Metayate
124         Berlin Met
Name: name, Length: 140, dtype: object
lat id
246     13.57384
248    14.184292
144     14.09999
249    13.829981
252    13.621025
         ...    
112       13.787
264      13.6836
255       13.697
257      14.1421
124      13.5072
Name: lat, Length: 140, dtype: object
lon id
246    -89.837845
248    -89.516655
144       -89.417
249    -89.437325
252    -89.263985
          ...    
112      -87.8354
264      -89.1948
255       -89.938
257      -89.2131
124      -88.5064
Name: lon, Length: 140, dtype: object



NameError: ignored

'01/06/2021 08:00 AM'

CLEAN

In [112]:
# ------------------------------------
# -------------- imports -------------
# ------------------------------------
import psycopg2 
from psycopg2 import Error
from pprint import pprint 
import requests
import json
import pandas as pd
import re 

# ------------------------------------
# ------------- functions ------------
# ------------------------------------
def insert_station(station_name, latitude, longitude, source_station_id, original_id):
  source_id = 4 # ETESA - HIDROMET PA
  query = """
    insert into station (source_id,station_name,latitude,longitude, source_station_id, original_id) 
      values ( '{}', '{}', '{}', '{}', '{}', '{}') on conflict do nothing;
    """.format(source_id, station_name, latitude, longitude, source_station_id, original_id)
  return query
  #cursor.execute(query)
  #pprint(cursor.fetchall())

def insert_data_all(variable_id, data_date, data_value, source_station_id):
  query = """
    INSERT INTO data_all(variable_id,data_date,data_value, station_id)
      VALUES({},'{}','{}',{}) on conflict do nothing;
    """.format(
        variable_id, data_date, data_value,
        "(SELECT id FROM station WHERE source_station_id = '{}')".format(source_station_id)
        )
  return query


# manipulate data functions
def get_estaciones(option = "JSON"):
  queryParams = {
      'option': option,
      'key': "098f6bcd4621d373cade4e832627b4f6",
  }
  response = requests.get("https://www.snet.gob.sv/dataservices/getEstaciones.php", params=queryParams)
  #print(response.json)
  data = json.loads(response.text)
  #data = data["sensores"]
  #pprint(data)

  #getting column names
  #variables = data["features"][1].keys()
  #print(variables)

  df = pd.DataFrame(data)
  df.rename(columns = {0:'id',1:'name',2:'lat',3:'lon',}, inplace = True)
  #display(df)
  return df;

# manipulate data functions
def get_data(option = "JSON"):
  queryParams = {
      'option': option,
      'key': "098f6bcd4621d373cade4e832627b4f6",
  }
  response = requests.get("https://www.snet.gob.sv/dataservices/getData.php", params=queryParams)
  #print(response.json)
  data = json.loads(response.text)
  #data = data["sensores"]
  #pprint(data)

  #getting column names
  #variables = data["features"][1].keys()
  #print(variables)

  df = pd.DataFrame(data)
  df.rename(columns = {0:'id_estacion',1:'param',2:'datetime',3:'valor',}, inplace = True)
  #display(df)
  return df;

# manipulate data functions
def get_parametros(option = "JSON"):
  queryParams = {
      'option': option,
      'key': "098f6bcd4621d373cade4e832627b4f6",
  }
  response = requests.get("https://www.snet.gob.sv/dataservices/getParametros.php", params=queryParams)
  #print(response.json)
  data = json.loads(response.text)
  #data = data["sensores"]
  #pprint(data)

  #getting column names
  #variables = data["features"][1].keys()
  #print(variables)

  df = pd.DataFrame(data)
  #df.rename(columns = {0:'id',1:'name',2:'lat',3:'lon',}, inplace = True)
  #display(df)
  return df;

# ------------------------------------
# ------------ connection ------------
# ------------------------------------  
connection = None
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="climahub",
                                  password="climahub$2021",
                                  host="138.197.98.178",
                                  port="5432",
                                  database="climahub")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("🟢 CONNECTION SUCCESFUL: PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("❌ Error while connecting to PostgreSQL", error)
  
# --------------------------------------------------------------------------
# -------------- get estaciones - data & insert ----------------------------
# --------------------------------------------------------------------------
dfES = get_estaciones();
dfES.set_index("id", inplace=True)
dfDA = get_data();
variables = {
  'PC': ["PCP", 5],
  'PP': ["PP", 17],
  'HG': ["LEV", 13],
  'AT': ["TMP", 3],
  'BP': ["PRS", 10], 
  'DA': ["WND", 7],
  #'DI': ["PCP", 5],
  #'DR': ["PCP", 5],
  'HR': ["HSOL", 14],
  #'LH': ["HRP", 15],
  #'LT': ["PCP", 5],
  'RA': ["WNG", 9],
  #'RD': ["PCP", 5],
  'RH': ["HRP", 15],
  'SA': ["WNS", 8],
  #'SI': ["PCP", 5],
  'SR': ["RSOL", 16],
  #'UH': ["PCP", 5],
  #'UT': ["PCP", 5],
  #'BA': ["PCP", 5],
  'DP': ["DWP", 4],
  #'QI': ["PCP", 5],
  #'PD': ["PCP", 5],
  #'RI': ["PCP", 5],
  #'SO': ["PCP", 5],
  #'MA': ["PCP", 5],
  #'SM': ["PCP", 5],
  #'LW': ["PCP", 5],
  # 'ST': ["PCP", 5],
}
GRAND_QUERY = ''
for index, row in dfES.iterrows():
    #print(index,' - ',row['name'], '(', row['lat'],', ',row['lon'], ')' )
    GRAND_QUERY+=insert_station(row['name'], row['lat'], row['lon'], index, index)

#print(GRAND_QUERY)
cursor.execute(GRAND_QUERY)
connection.commit()

GRAND_QUERY = ''
for index, row in dfDA.iterrows():
  if(row['param'] != "  " and row['param'] in variables):
    #print(index,' - ',row['id_estacion'],' - ',row['param'],' - ',row['datetime'],' - ',row['valor'])
    GRAND_QUERY+=insert_data_all(variables[row['param']][1],row['datetime'], row['valor'],row['id_estacion'] )
    
#print(GRAND_QUERY)
cursor.execute(GRAND_QUERY)
connection.commit()

cursor.close()
connection.close()
print("PostgreSQL connection is closed")

🟢 CONNECTION SUCCESFUL: PostgreSQL server information
{'user': 'climahub', 'dbname': 'climahub', 'host': '138.197.98.178', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.2 (Debian 13.2-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',) 

PostgreSQL connection is closed
